In [ ]:
import sys
sys.path.append('..')
import elastic_rods
import numpy as np
from typing import NamedTuple
from bending_validation import suppress_stdout as so

In [ ]:
npts = 100

In [ ]:
t = np.linspace(0, 2 * np.pi, npts-2, endpoint=False)
t = np.concatenate([t, t[0:2]])

In [ ]:
R = 40
pts = np.column_stack([R * np.cos(t), R * np.sin(t), np.zeros_like(t)])

In [ ]:
pr = elastic_rods.PeriodicRod(pts, True)
pr.setMaterial(elastic_rods.RodMaterial('rectangle', 4.0e10 / 1e6, 0.3, [1, 1], stiffAxis=elastic_rods.StiffAxis.D2))

In [ ]:
dc = pr.rod.deformedConfiguration()

In [ ]:
Hsp = pr.hessianSparsityPattern(1.0)

In [ ]:
import linkage_vis, vis
import matplotlib

In [ ]:
import time
def updateView():
    v.update()
    time.sleep(0.01)
    cv2.update(vectorField=np.array([d.d2 for d in dc.materialFrame]))
    refVF = vis.fields.VectorField(cv.mesh, np.array([d.d2 for d in dc.referenceDirectors]), colormap=matplotlib.cm.cool_r, glyph=vis.fields.VectorGlyph.CYLINDER)
    cv.update(vectorField=refVF)

In [ ]:
v = linkage_vis.LinkageViewer(pr, width=1200, height=720)
cv = linkage_vis.CenterlineViewer(pr.rod, superView=v, vectorField=np.array([d.d2 for d in dc.referenceDirectors]))
cv2 = linkage_vis.CenterlineViewer(pr.rod, superView=v, vectorField=np.array([d.d2 for d in dc.materialFrame]))
v.show()

In [ ]:
pr.twist = 16 * np.pi

with so(): elastic_rods.compute_equilibrium(pr, fixedVars=[pr.numDoF()-1, pr.thetaOffset(), 0, 1, 2, 3, 4, 5], callback=lambda prob, i: updateView())

# Finite Difference Validation

In [ ]:
import fd_validation

In [ ]:
class FDWrapper:
    def __init__(self, rod):
        self.rod = rod
    def numVars(self): return self.rod.numDoF()
    def getVars(self): return self.rod.getDoFs()
    def setVars(self, x): return self.rod.setDoFs(x)
    def energy(self): return self.rod.energy()
    def gradient(self): return self.rod.gradient(updatedSource=False)
    def hessian(self): return self.rod.hessian()

In [ ]:
pr.setDoFs(pr.getDoFs() + 1e-3 * fd_validation.genPerturbation(pr.getDoFs()))
pr.rod.updateSourceFrame()

In [ ]:
fd_validation.gradConvergencePlot(FDWrapper(pr))

In [ ]:
fd_validation.hessConvergencePlot(FDWrapper(pr.rod))

In [ ]:
wpr = FDWrapper(pr)
fd_validation.hessConvergencePlot(wpr, perturb=fd_validation.basisDirection(wpr, 0))